In [1]:
#input
import os

#copy the link of the book's coverpage
coverpage_url = "https://sls.hkpl.gov.hk/digital-collection/tc/contentcoverpage.html?catalogueId=70a3b1d16b084fa0806deece0eeec0ed&mainKeyword=&currentPosition=1&searchId=f85e1994980a459d96a17ce9911a329d"

#destination folder
download_folder = os.getcwd()  #(or r"C:\downloads\dse past papers\math" etc.)

In [2]:
import re
import requests

#this is the catalogue id of the book
id = re.search("(?<=catalogueId=)[^&]*",coverpage_url)[0]
print(f"'{coverpage_url}'","\n")
print("catalogue id:\t",id)


#this link looks up the file number of the pdf with catalogue id
detail_url = "https://sls.hkpl.gov.hk/api/drmapi/client-api/mmis/catalogueDetail/catalogueDetail"
headers = {"catalogueId":id}

#send a request for resource details
s = requests.session()
r1 = s.get(detail_url,headers=headers)
data = r1.json()["data"]

#print resource title
print("resource name:\t", data["titleStr"])

#ids and file names of pdf files if any
#there should be only one pdf, but play safe
fileid = [(item["id"], item["fileName"]) for item in data["contentList"]["ortherContent"] if ".pdf" in item["fileName"]]


#this link downloads files with file id
download_url = "https://media.hkpl.gov.hk/api/drmapi/client-api/common/pdfCommon/load?id="
headers2 = {"Accept": "*/*","Accept-Encoding": "gzip, deflate, br, zstd",
          "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6",
         "Connection": "keep-alive","Host": "media.hkpl.gov.hk",
         "Origin": "https://sls.hkpl.gov.hk", "Referer": "https://sls.hkpl.gov.hk/"
        }

if len(fileid) > 0:
    print("\npdf found")
    for file in fileid:
        print(f"file id: {file[0]} (file name: {file[1]})")
        
    response = input("Enter 'Y' or 'y' to download all files\t\t")
    
    if response.strip() in ["Y","y"]:
        print()
        print("destination folder:\t", download_folder)
        for file in fileid:
            print("\ndownloading <"+file[1]+">")
            r2 = s.get(download_url+file[0],headers=headers2)
            
            # we use the default file name, change file[1] to some other file name if needed
            with open(download_folder+"\\"+file[1],"wb") as filehandle:
                filehandle.write(r2.content)
            print(download_folder+"\\"+file[1],"\ndownloaded <"+file[1]+">")
        print("\ndone")
else:
    print("\npdf not found")
    #pass

'https://sls.hkpl.gov.hk/digital-collection/tc/contentcoverpage.html?catalogueId=70a3b1d16b084fa0806deece0eeec0ed&mainKeyword=&currentPosition=1&searchId=f85e1994980a459d96a17ce9911a329d' 

catalogue id:	 70a3b1d16b084fa0806deece0eeec0ed
resource name:	 香港中學文憑考試試題專輯. 數學 必修部分2021

pdf found
file id: 1922965213401464833 (file name: HKDSE20210311_01.pdf)
Enter 'Y' or 'y' to download all files		y

destination folder:	 C:\Users\lenovo\python jupyter notebooks\web scraping\hkpl past papers

downloading <HKDSE20210311_01.pdf>
C:\Users\lenovo\python jupyter notebooks\web scraping\hkpl past papers\HKDSE20210311_01.pdf 
downloaded <HKDSE20210311_01.pdf>

done
